# Step :- 1 import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import os

# Step 2:- use VGG16 as a feature extractor

In [2]:
# load pre trained VGG16 model (excluding the top layers)
vgg = VGG16(input_shape=(150, 150, 3), weights='imagenet', include_top=False)

# Freeze all convolutional layers
for layers in vgg.layers:
  layers.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


# Step :- Data Preprocessing and Augmentation

In [3]:
!mkdir -p /content/data/train/cat
!mkdir -p /content/data/train/dog
!mkdir -p /content/data/validation/cat
!mkdir -p /content/data/validation/dog

In [4]:
# #for delete the directory's

# import os
# import shutil

# shutil.rmtree('/content/data/train/dog')

In [5]:
data_dir = '/content/data/train'
val_dir = '/content/data/validation'

In [6]:
train_dir = '/content/drive/MyDrive/Dataset_VGG16/dogs-vs-cats'

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalize image pixels from [0,255] to [0,1]
    validation_split=0.2,          # Split 20% of the data for validation
    rotation_range=40,            # Randomly rotate images up to 40 degrees
    width_shift_range=0.2,        # Randomly shift images horizontally by 20%
    height_shift_range=0.2,       # Randomly shift images vertically by 20%
    shear_range=0.2,              # Shear images (tilt them along axis)
    zoom_range=0.2,               # Randomly zoom into images by up to 20%
    horizontal_flip=True          # Randomly flip images horizontally
)

train_generator = train_datagen.flow_from_directory(
    train_dir,                    # Path to folder containing /cats and /dogs subfolders
    target_size=(150,150),       # Resize all images to 150x150 pixels
    batch_size=32,               # Load 32 images per batch
    class_mode='categorical',         # Because it’s a binary classification (cats vs dogs)
    subset='training'            # Use 80% of data for training
)

val_generator = train_datagen.flow_from_directory(
    train_dir,                    # Same directory as training
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'          # Use 20% of data for validation
)



Found 1218 images belonging to 2 classes.
Found 304 images belonging to 2 classes.


In [8]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     validation_split=0.2,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Explicitly define classes to ensure the generator expects 2 classes
# classes = ['cats', 'dogs'] # Adjust these class names based on your directory structure


# train_generator = train_datagen.flow_from_directory(
#     data_dir,
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='categorical',
#     subset='training',
#     classes= classes
# )

# val_generator = train_datagen.flow_from_directory(
#     data_dir,
#     target_size=(150, 150),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation',
#     classes=classes
# )
# #print class indices to verify that th generator is finding the correct number of classses
# print("class Indices:", train_generator.class_indices)

#step 4:- make classifier model

In [9]:
# add VGG16 into Sequential model and add over custom dense layers
model = Sequential([
    vgg,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# step 5:- Train and compile the model

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    batch_size = 64
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 286s 7s/step - accuracy: 0.9001 - loss: 0.1693 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 286s 7s/step - accuracy: 1.0000 - loss: 1.2482e-06 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 262s 7s/step - accuracy: 1.0000 - loss: 1.1890e-06 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 266s 7s/step - accuracy: 1.0000 - loss: 4.8628e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 290s 7s/step - accuracy: 1.0000 - loss: 4.0997e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 265s 7s/step - accuracy: 1.0000 - loss: 5.0226e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 263s 7s/step - accuracy: 1.0000 - loss: 5.7268e-07 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 281s 7s/step - accura

In [11]:
loss, acc = model.evaluate(val_generator)
print("Test Accuracy:", acc)
print("Test Loss: ", loss)

10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.0
Test Loss:  0.0
